In [1]:
import os
import math
from datetime import datetime, timedelta
from dotenv import load_dotenv
import pymysql

# Load environment variables
load_dotenv()

# Database configuration
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEMA = os.getenv("DB_SCHEMA")

# Connect to the database
con = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, port=3306, database=DB_SCHEMA)

# Configuration
data = []
month = 2  # February
days = 28  # Number of days in the month
year = 2025

# Define working hours threshold
WORKING_HOURS_THRESHOLD_EARLY = 8  # Standard working hours if punch-in before 07:30
WORKING_HOURS_THRESHOLD_LATE = 9  # Standard working hours if punch-in after 07:30
CUTOFF_TIME = datetime.strptime("07:30:00", "%H:%M:%S").time()

# Prepare headers with separate OT columns
headers = ["Emp Id", "Name", "Deploy Site", "Employee Type"]
for i in range(1, days + 1):
    date_str = datetime(year, month, i).date().strftime('%d %B')
    headers.append(date_str)  # Punch-in/out column
    headers.append(f"OT ({date_str})")  # OT column
headers.extend(["Total Days", "Total OT"])

# Fetch employee data
with pymysql.cursors.DictCursor(con) as cursor:
    cursor.execute("SELECT * FROM employees")
    employees = cursor.fetchall()

    for employee in employees:
        # Initialize employee record in data
        employee_data = {
            "Emp Id": employee['empId'],
            "Name": employee['employeeName'],
            "Deploy Site": employee['employeeDeploySite'],
            "Employee Type": employee['employeeTypeId'],
            "Total Days": 0,
            "Total OT": 0
        }

        # Initialize OT tracking
        total_ot = 0

        # Fetch attendance records with daily OT for each day
        with pymysql.cursors.DictCursor(con) as attCursor:
            attCursor.execute(
                "SELECT * FROM attendance_record WHERE emp_id = %s AND date BETWEEN %s AND %s ORDER BY date",
                (employee['empId'], f"{year}-{month:02d}-01", f"{year}-{month:02d}-31")
            )
            attendances = attCursor.fetchall()

            for attendance in attendances:
                date_key = attendance["date"].strftime("%d %B")
                ot_key = f"OT ({date_key})"

                # Extract punch-in and punch-out times
                punch_in_time = attendance['punch_in'][11:19] if attendance['punch_in'] else "NaN"
                punch_out_time = attendance['punch_out'][11:19] if attendance['punch_out'] else "NaN"
                ot_hours = 0  # Default OT hours

                # Handle "NPO" for punch-out times
                if punch_out_time in ["22:00:00", "10:00:00"]:
                    punch_out_time = "NPO"
                    ot_hours = "NPO"
                elif attendance['punch_in'] and attendance['punch_out']:
                    punch_in = datetime.strptime(punch_in_time, "%H:%M:%S")
                    punch_out = datetime.strptime(punch_out_time, "%H:%M:%S")

                    # Handle overnight shifts
                    if punch_out < punch_in:
                        punch_out += timedelta(days=1)

                    # Calculate working hours correctly
                    working_hours = (punch_out - punch_in).total_seconds() / 3600
                    working_hours_threshold = WORKING_HOURS_THRESHOLD_LATE if punch_in.time() > CUTOFF_TIME else WORKING_HOURS_THRESHOLD_EARLY

                    # Mark "ABSENT" if working hours < 2
                    # Calculate OT if working hours exceed the threshold
                    if working_hours > working_hours_threshold:
                        ot_hours = round(working_hours - working_hours_threshold, 2)
                    else:
                        ot_hours = 0  # Ensure OT is 0 when below threshold

                # Store data
                employee_data[date_key] = f"{punch_in_time}\n{punch_out_time}\nOT: {ot_hours}"
                if isinstance(ot_hours, (int, float)):  # Only add numeric OT values to the total
                    total_ot += ot_hours

        # Fetch attendance history for total days
        with pymysql.cursors.DictCursor(con) as attCursor2:
            attCursor2.execute(
                "SELECT * FROM attendance_history WHERE emp_id = %s AND year = %s AND month = %s",
                (employee['empId'], year, month)
            )
            history = attCursor2.fetchone()

            if history:
                employee_data["Total Days"] = math.ceil(history['days'])
                employee_data["Total OT"] = total_ot

        data.append(employee_data)

# Close the database connection
con.close()

# Now `data` contains:
# - Daily OT values
# - "NPO" for invalid Punch-Out times
# - "ABSENT" if working hours < 2
# - Proper handling of overnight shifts


In [2]:
import pandas as pd

df = pd.DataFrame(data)

for header in headers:
    if not header in df.columns:
        df[header] = None

df = df[headers]
df.to_csv("newoutput.csv", index=False)